In [2]:
import os,shutil
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

        
class data_f(object):
    def __init__(self):
        self.IP = []
        self.cert = []
        self.urls = []
        self.static_features = []
        self.seq = []
        self.label = []
        self.time = []
        self.image = []
        self.fn = []
        
    def insert(self,IP,cert,urls,static_features,seq,label,time,image,fn):
        self.IP.append(IP)
        self.cert.append(cert)
        self.urls.append(urls)
        self.static_features.append(static_features)
        self.seq.append(seq)
        self.label.append(label)
        self.time.append(time)
        self.image.append(image)
        self.fn.append(fn)

In [3]:
from interval import Interval
import os,shutil
import numpy as np
import pickle


class cert_cluster(object):
    def __init__(self):
        self.IPs = []
        self.urls = {} #{urls:number}
        self.url_number = [] # sorted (url:number)
        self.url_weight = {}#{url:weight}
        self.url_unique = []
        self.certs = [] 
        self.sessions = []
        self.images = []
        self.static_features = []
        self.fn = []
        

        self.seq = []
        self.label = -1
        self.time_slides = [] #(start,end)
        
        self.sim_list = []
        self.url_all = []
        
    #file name
    def fn_cal(self):
        self.fn = [ss['fn'] for ss in self.sessions]
            
        
    #all urls
    def all_url_cal(self):
        for session in self.sessions:
            urls = session['urls']
            for url in urls:
                if url not in self.url_all:
                    self.url_all.append(url)
    #url sim cal
    def sim_cal(self,urls_dict):
        self.sim_list = []
        for url_dict_per in urls_dict:
            score = 0
            for url in self.url_unique:
                if url in url_dict_per:
                    score += 1
            self.sim_list.append(score / (len(self.url_unique)*1.0))
        if np.array(self.sim_list).max() > 0:
            self.sim_list = self.sim_list/np.array(self.sim_list).max()
        max_idx = np.argmax(np.array(self.sim_list))
        for i in range(len(self.sim_list)):
            if i!=max_idx:
                self.sim_list[i] = 0
            
    #ip sim cal
    def sim_ip_cal(self,urls_dict):
        self.sim_list = []
        for url_dict_per in urls_dict:
            score = 0
            for url in self.IPs:
                if url in url_dict_per:
                    score += 1
            self.sim_list.append(score)
        if np.array(self.sim_list).max() > 0:
            self.sim_list = self.sim_list/np.array(self.sim_list).max()
        max_idx = np.argmax(np.array(self.sim_list))
        for i in range(len(self.sim_list)):
            if i!=max_idx:
                self.sim_list[i] = 0
        
            
    
    #static
    def static_cal(self):
        self.static_features = [a['static_features'] for a in self.sessions]
    
    
    #major voting for cluster's label
    def label_cal(self):
        labels = [a['label'] for a in self.sessions]
        self.label = max(labels,key = labels.count)
    
    #image extract
    def image_cal(self):
        self.images = [item['image'].flatten() for item in self.sessions]
    
    #seq_ipdaate
    def seq_cal(self):
#         self.seq = [item['seq'].flatten() for item in self.sessions]
        for i in range(len(self.sessions)):
            session = self.sessions[i]
            mat = session['seq']
            sta = session['static_features']
            for item in sta:
                mat = np.append(mat,float(item))

            self.seq.append(mat)
            
            
        
    #time_slieds generation
    def time_cal(self):
        def getfirst(item):
            return item[0]
        for session in self.sessions:
            self.time_slides.append(session['time'])
        self.time_slides = sorted(self.time_slides,key=getfirst)
        
        #concat time period
        self.time_slides = [Interval(item[0],item[1],lower_closed=True, upper_closed=True) for item in self.time_slides]

        while True:
            big_flag = 0
            for i in range(len(self.time_slides)):
                
                flag = 0
                for j in range(len(self.time_slides)):

                    if i ==len(self.time_slides)-1 and j == len(self.time_slides)-1:
                        big_flag =1
                    if i==j:
                        continue      
                    if self.time_slides[i].overlaps(self.time_slides[j]):
                        interval_merge = self.time_slides[i].join(self.time_slides[j])
                        self.time_slides[i] = interval_merge
                        del self.time_slides[j]

                        flag = 1
                        break                
                if flag == 1:
                    break
            if big_flag==1:
                break
#         print(len(self.time_slides))
        
                    
                    
                
            
            
        
      
    #high-frequency urls calcualte
    def urls_cal(self):
        for item in self.sessions:
            urls = item['urls']
            for url in urls:
                if url not in list(self.urls.keys()):
                    self.urls[url] = 1
                else:
                    self.urls[url] += 1
        #sorted
        self.url_number = sorted(self.urls.items(),key=lambda item:item[1],reverse=True)
    
    #url importance score
    def urls_weight(self):
        total = np.array([item[1] for item in self.url_number]).sum()*1.0
        for item in self.url_number:
            self.url_weight[item[0]] = item[1]*1.0/total
                
#         self.url_weight = [(item[0],item[1]*1.0/total) for item in self.url_number]
        self.url_unique = [item[0] for item in self.url_number]
        
    #IP summarize
    def ips_cal(self):
        for item in self.sessions:
            if item['IP'] not in self.IPs:
                self.IPs.append(item['IP'])
    
    #cert summarize
    def cert_cal(self):
        for item in self.sessions:
            if item['cert'] not in self.certs:
                self.certs.append(item['cert'])
    
    #discard low-frequency urls
    def url_clean(self):
        if len(self.url_number) <=5:
            return
        self.url_number = self.url_number[:5]
        self.url_unique = [item[0] for item in self.url_number]
        self.url_weight = {}
        self.urls_weight()
        
        
    
    #update cert_clt
    def update(self):
        self.urls_cal()
        self.urls_weight()
        self.ips_cal()
        self.cert_cal()
        self.seq_cal()
        self.image_cal()
        self.label_cal()
        self.static_cal()
        self.url_clean()
        self.all_url_cal()
        self.fn_cal()



In [19]:
def clustering(filename):

    data_list = []
    dataT = data_f()
    with open(filename,'rb') as file:
        dataT  = pickle.loads(file.read())

    for i in range(len(dataT.label)):
        tmp = {}
        tmp['IP'] = dataT.IP[i]
        tmp['cert'] = dataT.cert[i]
        tmp['urls'] = dataT.urls[i]
        tmp['static_features'] = dataT.static_features[i]
        tmp['seq'] = dataT.seq[i]
        tmp['label'] = dataT.label[i]
        tmp['time'] = dataT.time[i]
        tmp['image'] = dataT.image[i]
        tmp['fn'] = dataT.fn[i]

        data_list.append(tmp)


    #cert clusterting
    cert_clusters = []
    used_sessions = []
    for i in range(len(data_list)):
        session = data_list[i]
        cert = session['cert']
        if cert is 0:
            continue
        if len(cert_clusters) is 0:
            cert_clt = cert_cluster()
            cert_clt.sessions.append(session)
            cert_clt.cert_cal()
            cert_clusters.append(cert_clt)

            continue
        judge = 0
        for j in range(len(cert_clusters)):

            if cert in cert_clusters[j].certs:
                cert_clusters[j].sessions.append(session)
                judge  = 1
        if judge is 0:
            cert_clt = cert_cluster()
            cert_clt.sessions.append(session)
            cert_clt.cert_cal()
            cert_clusters.append(cert_clt)

        used_sessions.append(i)

    #update cert_clusters
    for j in range(len(cert_clusters)):
        cert_clusters[j].update()
    #update the remaining sessions in session list
    data_list = [data_list[i] for i in range(len(data_list)) if i not in used_sessions]
#     print(len(data_list)) 


    #cert clt clustering
    #merge the root cluster pairs with same urls
    while len(cert_clusters) >10:
        matching_matrix = np.zeros((len(cert_clusters),len(cert_clusters)))
        #matching graph construct
        for i in range(len(cert_clusters)):
            row_clt = cert_clusters[i]
            for j in range(len(cert_clusters)):
                if i == j :
                    continue

                column_clt = cert_clusters[j]
                r2c = 0
                for url_row in row_clt.url_unique:
                    if url_row in column_clt.url_unique:
                        r2c += column_clt.url_weight[url_row]
                matching_matrix[i][j] = r2c

        #merge the root cluster pairs with same urls
        matching_matrix += matching_matrix.T
        matching_matrix = np.triu(matching_matrix).flatten()
        max_index = np.argmax(matching_matrix)
        i = int(max_index / (len(cert_clusters)))
        j = max_index%(len(cert_clusters))

        if i==j:
            break
        #merge
        cert_clusters[i].sessions.extend(cert_clusters[j].sessions)
        #更update root clusters clt
        cert_clusters[i].update()
        del cert_clusters[j]

    #gather ip to cert
    used_sessions = []
    for i in range(len(data_list)):
        session = data_list[i]
        ip = session['IP']

        for j in range(len(cert_clusters)):

            if ip in cert_clusters[j].IPs:
                cert_clusters[j].sessions.append(session)
                used_sessions.append(i)

                #update features in cert_clusters
                cert_clusters[j].update()
                break

    #update the remaining sessions in session list
    data_list = [data_list[i] for i in range(len(data_list)) if i not in used_sessions]
#     print(len(data_list))


    #urls
    for i in range(len(cert_clusters)):
        cert_clusters[i].url_clean()

    #concrete matching

    from fuzzywuzzy import fuzz
    
    used_sessions = []
    for i in range(len(data_list)):
        session = data_list[i]
        urls = session['urls']
        if len(urls) is 0:
            continue

        cert_clt_can = -1
        can_max = 0
        for j in range(len(cert_clusters)):
            cert_clt = cert_clusters[j]

            match_per = 0
            for url in urls:
                if url in cert_clt.url_unique:
                    match_per+=1
            if match_per > can_max:
                can_max = match_per
                cert_clt_can = j

        if cert_clt_can is -1:
            continue

        cert_clusters[cert_clt_can].sessions.append(data_list[i])

        #update cert_clusters
        cert_clt.update()

        used_sessions.append(i)

    data_list = [data_list[i] for i in range(len(data_list)) if i not in used_sessions]
#     print(len(data_list))

    for i in range(len(cert_clusters)):
        cert_clusters[i].url_clean()



    while len(cert_clusters) >10:
        matching_matrix = np.zeros((len(cert_clusters),len(cert_clusters)))

        for i in range(len(cert_clusters)):
            row_clt = cert_clusters[i]
            for j in range(len(cert_clusters)):
                if i == j :
                    continue

                column_clt = cert_clusters[j]
                r2c = 0
                for url_row in row_clt.url_unique:
                    if url_row in column_clt.url_unique:
                        r2c += column_clt.url_weight[url_row]
                matching_matrix[i][j] = r2c


        matching_matrix += matching_matrix.T
        matching_matrix = np.triu(matching_matrix).flatten()
        max_index = np.argmax(matching_matrix)
        i = int(max_index / (len(cert_clusters)))
        j = max_index%(len(cert_clusters))

        if i==j:
            break

        cert_clusters[i].sessions.extend(cert_clusters[j].sessions)

        cert_clusters[i].update()

        del cert_clusters[j]


    url_clusters = []
    used_sessions = []

    #inital url clusters
    for i in range(len(data_list)):
        session = data_list[i]
        if len(session['urls']) == 0:
            continue
        used_sessions.append(i)
        clt = cert_cluster()
        clt.sessions.append(session)
        clt.update()
        url_clusters.append(clt)

    while True:
        matching_matrix = np.zeros((len(url_clusters),len(url_clusters)))

        for i in range(len(url_clusters)):
            row_clt = url_clusters[i]
            for j in range(len(url_clusters)):
                if i == j :
                    continue

                column_clt = url_clusters[j]
                r2c = 0
                for url_row in row_clt.url_unique:
                    if url_row in column_clt.url_unique:
                        r2c += column_clt.url_weight[url_row]
                matching_matrix[i][j] = r2c


        matching_matrix += matching_matrix.T
        matching_matrix = np.triu(matching_matrix).flatten()
        max_index = np.argmax(matching_matrix)
        i = int(max_index / (len(url_clusters)))
        j = max_index%(len(url_clusters))

        if i==j:
            break

        url_clusters[i].sessions.extend(url_clusters[j].sessions)

        url_clusters[i].update()

        del url_clusters[j]

    data_list = [data_list[i] for i in range(len(data_list)) if i not in used_sessions]



    clusters = []
    clusters.extend(cert_clusters)
    clusters.extend(url_clusters)

    used_sessions = []
    for i in range(len(data_list)):
        session = data_list[i]
        ip = session['IP']

        for j in range(len(clusters)):
    #         cert_clt = clusters[j]
            if ip in clusters[j].IPs:
                clusters[j].sessions.append(session)
                used_sessions.append(i)

                #update cert_clusters
                clusters[j].update()
                break

    #update the remaining sessions in session list
    data_list = [data_list[i] for i in range(len(data_list)) if i not in used_sessions]
#     print(len(data_list))

    IP_clusters = []
    used_sessions = []

    #inital IP clusters
    for i in range(len(data_list)):
        session = data_list[i]
        IP = session['IP']
        if len(IP_clusters) == 0:
            used_sessions.append(i)
            clt = cert_cluster()
            clt.sessions.append(session)
            clt.update()
            IP_clusters.append(clt)
        else:
            d = 0
            for j in range(len(IP_clusters)):
                if IP in IP_clusters[j].IPs:
                    IP_clusters[j].sessions.append(session)
                    IP_clusters[j].update()
                    used_sessions.append(i)
                    d = 1
                    break

            if d == 0:
                used_sessions.append(i)
                clt = cert_cluster()
                clt.sessions.append(session)
                clt.update()
                IP_clusters.append(clt)

    data_list = [data_list[i] for i in range(len(data_list)) if i not in used_sessions]
#     print(len(data_list))

    clusters.extend(IP_clusters)

    sorted_clusters = sorted(clusters,key=lambda item:len(item.sessions),reverse=False)

    while len(sorted_clusters) > 10:
        judge = 0
        sorted_clusters = sorted(sorted_clusters,key=lambda item:len(item.sessions),reverse=False)
        for i in range(len(sorted_clusters)):
            judge = 0
            match_num = 0
            match_idx = -1
            for j in range(len(sorted_clusters)):
                if i==j:
                    continue
                match = len(set(sorted_clusters[i].url_unique)&set(sorted_clusters[j].url_unique))
                if match>match_num:
                    match_idx = j
                    match_num = match
                    judge =1


            if judge == 1:
                sorted_clusters[match_idx].sessions.extend(sorted_clusters[i].sessions)
                sorted_clusters[match_idx].update()

                del sorted_clusters[i]
                break
        if judge ==0:
            break


#     print(len(sorted_clusters))           


    dataT = data_f()

    with open("data/InitTraffic.pkl",'rb') as file:
        dataT  = pickle.loads(file.read())


    #ip dict   
    data_list = []

    for i in range(len(dataT.label)):
        tmp = {}
        tmp['IP'] = dataT.IP[i]
        tmp['cert'] = dataT.cert[i]
        tmp['urls'] = dataT.urls[i]
        tmp['static_features'] = dataT.static_features[i]
        tmp['seq'] = dataT.seq[i]
        tmp['label'] = dataT.label[i]
        tmp['time'] = dataT.time[i]
        tmp['image'] = dataT.image[i]
        tmp['fn'] = dataT.fn[i]

        data_list.append(tmp)
#     print(len(data_list))

# UMT construction
    clusters_dict = [cert_cluster() for i in range(10)]
    for item in data_list:
        idx = item['label']
        clusters_dict[idx].sessions.append(item)

    for i in range(len(clusters_dict)):
        clusters_dict[i].update()

    urls_dict = []                             
    for clt in clusters_dict:
        urls_dict.append(clt.IPs)

    for i in range(len(sorted_clusters)):
    #     with_urls_list[i].sim_cal(urls_dict)
        sorted_clusters[i].sim_ip_cal(urls_dict)

    #construct sim matrix
    while len(sorted_clusters)>10:
        matching_matrix = np.zeros((len(sorted_clusters),len(sorted_clusters)))

        for i in range(len(sorted_clusters)):
            row_clt = sorted_clusters[i]
            for j in range(len(sorted_clusters)):
                if i == j :
                    continue

                column_clt = sorted_clusters[j]

                if np.array(row_clt.sim_list).sum() == 0 or np.array(column_clt.sim_list).sum() == 0:
                    sim = 0.0
                else:
                    sim = cosine_similarity(np.array(row_clt.sim_list).reshape(1,-1),np.array(column_clt.sim_list).reshape(1,-1))

                matching_matrix[i][j] = sim


        matching_matrix += matching_matrix.T
        matching_matrix = np.triu(matching_matrix).flatten()
        max_index = np.argmax(matching_matrix)
        i = int(max_index / (len(sorted_clusters)))
        j = max_index%(len(sorted_clusters))

        if i==j:
            break

        sorted_clusters[i].sessions.extend(sorted_clusters[j].sessions)

        sorted_clusters[i].update()
        sorted_clusters[i].sim_ip_cal(urls_dict)

        del sorted_clusters[j]

    clusters_dict = [cert_cluster() for i in range(10)]
    for item in data_list:
        idx = item['label']
        clusters_dict[idx].sessions.append(item)

    for i in range(len(clusters_dict)):
        clusters_dict[i].update()

    urls_dict = []

    for i in range(len(clusters_dict)):
        clt = clusters_dict[i]
        urls_dict.append([item for item in clt.url_all])


    # sorted_cluster matching 
    # seperate sorted_cluster into 2 lists(with url and without url)
    with_urls_list = []
    wo_urls_list = []
    for clt in sorted_clusters:
        if len(clt.url_unique) == 0:
            wo_urls_list.append(clt)
        else:
            with_urls_list.append(clt)


    # we aim to cluster clts into 6 clts in  with_urls_list by urls_dict 
    for i in range(len(with_urls_list)):
        with_urls_list[i].sim_cal(urls_dict)


    #construct sim matrix
    while len(with_urls_list)>10:
        matching_matrix = np.zeros((len(with_urls_list),len(with_urls_list)))

        for i in range(len(with_urls_list)):
            row_clt = with_urls_list[i]
            for j in range(len(with_urls_list)):
                if i == j :
                    continue

                column_clt = with_urls_list[j]
                if np.array(row_clt.sim_list).sum() == 0 or np.array(column_clt.sim_list).sum() == 0:
                    sim = 0.0
                else:
                    sim = cosine_similarity(np.array(row_clt.sim_list).reshape(1,-1),np.array(column_clt.sim_list).reshape(1,-1))

                matching_matrix[i][j] = sim


        matching_matrix += matching_matrix.T
        matching_matrix = np.triu(matching_matrix).flatten()
        max_index = np.argmax(matching_matrix)
        i = int(max_index / (len(with_urls_list)))
        j = max_index%(len(with_urls_list))

        if i==j:
            break

        with_urls_list[i].sessions.extend(with_urls_list[j].sessions)

        with_urls_list[i].update()
        with_urls_list[i].sim_cal(urls_dict)

        del with_urls_list[j]

    return with_urls_list,wo_urls_list,urls_dict


In [20]:
import numpy as np
import math
def matching_process(G_a, G_b,c):
    '''
    Input: Two candidate graphs for matching with same shape. (N*N)
    Output: matching list of vertices from Graph A and Graph B.
    '''
    
    # matrix similarity

    def M_measure(As_a,As_b):
        '''
        Input: Assigment pair with shape of As_a =  (G_a_vertice, G_b_vertice)
        Output: M(As_a,As_b)
        '''
    
        A_src_index = As_a[0]
        A_dst_index = As_a[1]

        B_src_index = As_b[0]
        B_dst_index = As_b[1]
        
#         return math.exp(-(G_a[A_src_index][A_dst_index] - G_b[B_src_index][B_dst_index])**2 / (2*c*c))

        return math.exp(-(G_a[A_src_index][B_src_index] - G_b[A_dst_index][B_dst_index])**2 / (2*c*c))
    
    #construct Assigments list by travelling all vertice pairs. Length of N*N.
    As_list = []
    for i in range(G_a.shape[0]):
        for j in range(G_a.shape[1]):
            As_list.append((i,j))
#     print(As_list)
    
    #construct Assigment similarity by travelling all assigments in assigments list, with shape of (N2,N2).
    M = []
    for item_left in As_list:
        for item_right in As_list:
            M.append(M_measure(item_left,item_right))
#             print(item_left,item_right)
    
    M = np.array(M)
    M = M.reshape((G_a.shape[0]**2, G_a.shape[0]**2))
#     print(As_list)
#     print(M)
    
    # caculate principle eigenvector of M
#     print(M == M.T)
    eigenvalue,featurevector = np.linalg.eig(M)
    eigen_max_index = np.argmax(eigenvalue)
#     print(eigenvalue)
#     print(featurevector)
#     print(eigen_max_index)
    
    
    
    prl_eigenvector = featurevector.T[eigen_max_index]
    
    prl_eigenvector = prl_eigenvector.tolist()
#     print(prl_eigenvector)
    #sort principle eigenvector M by value
    sort_index_M = [(item,prl_eigenvector.index(item)) for item in prl_eigenvector]
#     print(sort_index_M)
    
    def get_first(item):
        return item[0]
    #sort_index_M is the sorted assigments index 
    sort_index_M = sorted(sort_index_M, key=get_first, reverse=True)
    sort_index_M = [item[1] for item in sort_index_M]
    
    #accepting and rejecting process
    As_list_accept_status = [0]*len(As_list)
    G_a_matched_index = []
    G_b_matched_index = []
    
    matching_list = [-1] * G_a.shape[0]
    for index in sort_index_M:
        match = As_list[index]
#         if  match[1] in G_b_matched_index:
#             continue
        if match[0] in G_a_matched_index or match[1] in G_b_matched_index:
            continue
        matching_list[match[1]] = match[0]
        
        G_a_matched_index.append(match[0])
        G_b_matched_index.append(match[1])
    
    
    return matching_list
        
def count_acc(y_real,y_pred):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(y_real.shape[0]):
        if y_real[i] == y_pred[i]:
            tp += 1
    
    
#     Precision = tp/(tp+fp)
#     Recall = tp/(tp+fn)
#     f1_score = 2*Precision*Recall/(Precision+Recall)
    Acc = (tp)/(y_real.shape[0])
    print("Overall Accuracy: {} ".format(Acc))
#     print(tp)    

In [21]:
from fuzzywuzzy import fuzz
import random
from simhash import Simhash

#dimension [urls, IP, static, seq, image]
def clt_sim(clt1,clt2,url_dict=None):
    sim_vec = [0]*1
    
    #urls simhash
#     if len(clt1.urls) == 0 or len(clt2.urls) == 0:
#         sim_vec[0] = 0
#     else:
#         num = 2
#         if len(clt1.url_unique)<num:
#             num = len(clt1.url_unique)
#         if len(clt2.url_unique)<num:
#             num = len(clt1.url_unique)
#         urls1 = clt1.url_unique[:2]
#         urls2 = clt2.url_unique[:2]
#         print(urls1)
#         print(urls2)
        
        
#         total = 0.0
#         for url1 in urls1:
#             for url2 in urls2:
# #                 total.append((fuzz.ratio(url1,url2)+fuzz.ratio(url2,url1))/200.0)
# #                 total.append(1.0 - Simhash(url1).distance(Simhash(url2))/100.0)
# #                 total += (fuzz.ratio(url1,url2)+fuzz.ratio(url2,url1)) /200.0
#                 total += 1.0 - Simhash(url1).distance(Simhash(url2))/100.0
        
#         total = total / (len(urls1) * len(urls2) *1.0)
# #         total = np.array(total).max()
#         print(total)
#         print()
#         sim_vec[0] = total
        
    #compare with dict
    if len(clt1.urls) == 0 or len(clt2.urls) == 0:
        sim_vec[0] = 0
    else:
        num = 2
        if len(clt1.url_unique)<num:
            num = len(clt1.url_unique)
        if len(clt2.url_unique)<num:
            num = len(clt1.url_unique)
        urls1 = clt1.url_unique[:2]
        urls2 = clt2.url_unique[:2]
    
    vec1 = np.zeros(10)
    for i in range(10):
        m = []
        for url in urls1:
            m1 = []
            for dict_url in url_dict[i]:
                sim = 1.0 - Simhash(url).distance(Simhash(dict_url))/100.0
                m1.append(sim)
#                 if sim >m1:
#                     m1 = sim
            m1 = np.array(m1).mean()
            m.append(m1)
#             if m1>m:
#                 m = m1
        vec1[i] = np.array(m).mean()
#         vec1[i] = m
    
    vec2 = np.zeros(10)
    for i in range(10):
        m = []
        for url in urls2:
            m1 = []
            for dict_url in url_dict[i]:
                sim = 1.0 - Simhash(url).distance(Simhash(dict_url))/100.0
                m1.append(sim)
#                 if sim >m1:
#                     m1 = sim
            m1 = np.array(m1).mean()
            m.append(m1)
#             if m1>m:
#                 m = m1
        vec2[i] = np.array(m).mean()
#     print(vec1)
#     print(vec2)
#     print()
    sim_vec[0] = cosine_similarity(vec1.reshape(1, -1),vec2.reshape(1, -1))
#     print(sim_vec[0])
        
        
    
        
#     #IP sim
#     IPs1 = clt1.IPs
#     IPs2 = clt2.IPs
        
#     total = 0.0
#     for IP1 in IPs1:
#         for IP2 in IPs2:
# #             total += (fuzz.ratio(IP1,IP2)+fuzz.ratio(IP2,IP1))/200.0
#             total += 1.0 - Simhash(IP1).distance(Simhash(IP2))/100.0
        
#     total = total / (len(IPs1) * len(IPs2) *1.0)
#     sim_vec[1] = total
    
    #static vector consine
#     total = 0.0
#     for f1 in clt1.static_features:
#         for f2 in clt2.static_features:
#             total += cosine_similarity(f1.reshape(1,-1),f2.reshape(1,-1))
#     total = total / (len(clt1.static_features) * len(clt2.static_features) *1.0)
#     sim_vec[2] = total[0][0]
    
# #     sim_vec[2] = cosine_similarity(np.array(clt1.static_features).mean(axis=0).reshape(1,-1),np.array(clt2.static_features).mean(axis=0).reshape(1,-1))
    
# #     seq consine
#     sim_vec[3] = cosine_similarity(np.array(clt1.seq).mean(axis=0).reshape(1,-1),np.array(clt2.seq).mean(axis=0).reshape(1,-1))[0][0]
#     total = 0.0
#     print((len(clt1.seq)))
#     print((len(clt2.seq)))
#     for f1 in clt1.seq:
#         for f2 in clt2.seq:
#             total += cosine_similarity(f1.reshape(1,-1),f2.reshape(1,-1))
#     total = total / (len(clt1.seq) * len(clt2.seq) *1.0)
#     sim_vec[3] = total
    
    #image consine
#     sim_vec[2] = cosine_similarity(np.array(clt1.images).mean(axis=0).reshape(1,-1),np.array(clt2.images).mean(axis=0).reshape(1,-1))[0][0]
#     total = 0.0
#     for f1 in clt1.images:
#         for f2 in clt2.images:
#             total += cosine_similarity(f1.reshape(1,-1),f2.reshape(1,-1))
#     total = total / (len(clt1.images) * len(clt2.images) *1.0)
#     sim_vec[0] = total[0][0]
    return  np.array(sim_vec).mean()
#     return sim_vec[0]

def clt_sim_same(clt1):
    sim_vec = [0]*1
    
    #urls
    if len(clt1.urls) == 0:
        sim_vec[0] = 0
    else:
        urls1 = clt1.url_unique[:2]
        
        total = 0.0
        for url1 in urls1:
            for url2 in urls1:
#                 total.append((fuzz.ratio(url1,url2)+fuzz.ratio(url2,url1))/200.0)
#                 total.append(1.0 - Simhash(url1).distance(Simhash(url2))/100.0)
#                 total += (fuzz.ratio(url1,url2)+fuzz.ratio(url2,url1))/200.0
                total += 1.0 - Simhash(url1).distance(Simhash(url2))/100.0
        
        total = total / (len(urls1) * len(urls1) *1.0)
#         total = np.array(total).max()
        sim_vec[0] = total
#         print(urls1)
#         print(total)
#         print()
    
    
        
    #IPs
#     IPs1 = clt1.IPs
        
#     total = 0.0
#     for IP1 in IPs1:
#         for IP2 in IPs1:
# #             total += (fuzz.ratio(IP1,IP2)+fuzz.ratio(IP2,IP1))/200.0
#             total += 1.0 - Simhash(IP1).distance(Simhash(IP2))/100.0
        
#     total = total / (len(IPs1) * len(IPs1) *1.0)
#     sim_vec[1] = total
    
#     #static vector consine
#     proto = np.array(clt1.static_features).mean(axis=0).reshape(1,-1)
#     total = 0.0
    
#     for static_features in clt1.static_features:
#         total += cosine_similarity(proto,np.array(static_features).reshape(1,-1))
#     sim_vec[2] =  total[0][0] / (len(clt1.static_features) *1.0)
    
#     #static vector consine
#     proto = np.array(clt1.seq).mean(axis=0).reshape(1,-1)
#     total = 0.0
    
#     for seq in clt1.seq:
#         total += cosine_similarity(proto,np.array(seq).reshape(1,-1))
#     sim_vec[3] =  total[0][0] / (len(clt1.seq) *1.0)
    
    
    #images
#     proto = np.array(clt1.images).mean(axis=0).reshape(1,-1)
#     total = 0.0
    
#     for image in clt1.images:
#         total += cosine_similarity(proto,np.array(image).reshape(1,-1))
#     sim_vec[0] =  total[0][0] / (len(clt1.images) *1.0)
    
    return np.array(sim_vec).mean()
        
    

In [22]:
def prepro(with_urls_list_normal,with_urls_list_poor):
    with_urls_list_normal_sorted = []

    for i in range(len(with_urls_list_normal)):
        for item in with_urls_list_normal:
            if item.label == i:
                with_urls_list_normal_sorted.append(item)

    with_urls_list_poor_sorted = []

    for i in range(len(with_urls_list_poor)):
        for item in with_urls_list_poor:
            if item.label == i:
                with_urls_list_poor_sorted.append(item)
    return with_urls_list_normal_sorted, with_urls_list_poor_sorted

In [23]:
def match_trans(with_urls_list_normal,with_urls_list_poor,urls_dict):
    matching_matrix_normal = np.zeros((len(with_urls_list_normal),len(with_urls_list_normal)))
    #construct SimGraph
    for i in range(len(with_urls_list_normal)):
        row_clt = with_urls_list_normal[i]
        for j in range(len(with_urls_list_normal)):
            column_clt = with_urls_list_normal[j]

            if i==j:
                sim = clt_sim_same(with_urls_list_normal[i])
            else:
                sim = clt_sim(with_urls_list_normal[i],with_urls_list_normal[j],urls_dict)

    #         sim = cosine_similarity(np.array(with_urls_list_normal[i].images).mean(axis=0).reshape(1,-1),np.array(with_urls_list_normal[j].images).mean(axis=0).reshape(1,-1))
            matching_matrix_normal[i][j] = sim
    #         print(sim)


    matching_matrix_poor = np.zeros((len(with_urls_list_poor),len(with_urls_list_poor)))
    #construct SimGraph
    for i in range(len(with_urls_list_poor)):
        row_clt = with_urls_list_poor[i]
        for j in range(len(with_urls_list_poor)):
            column_clt = with_urls_list_poor[j]
            if i==j:
                sim = clt_sim_same(with_urls_list_poor[i])
            else:
                sim = clt_sim(with_urls_list_poor[i],with_urls_list_poor[j],urls_dict)
    #         sim = cosine_similarity(np.array(with_urls_list_poor[i].images).mean(axis=0).reshape(1,-1),np.array(with_urls_list_poor[j].images).mean(axis=0).reshape(1,-1))
            matching_matrix_poor[i][j] = sim

    return matching_matrix_normal,matching_matrix_poor, matching_process(matching_matrix_normal,matching_matrix_poor, cosine_similarity(matching_matrix_normal.flatten().reshape(1, -1),matching_matrix_poor.flatten().reshape(1, -1)))


In [24]:
def clt_final(with_urls_list_poor,wo_urls_list_poor):
    def image_sim(clt1,clt2):
        total = 0.0
        for f1 in clt1.images:
            for f2 in clt2.images:
                total += cosine_similarity(f1.reshape(1,-1),f2.reshape(1,-1))
        total = total / (len(clt1.images) * len(clt2.images) *1.0)
        return total

    def time_sim(clt1,clt2):
        total = 0
        for time_row in clt1.time_slides:
            for time_col in clt2.time_slides:
                if time_row.overlaps(time_col):
                    r2c += 1
        return total


        for i in range(len(clt1)):
            row_clt = clt[i]
            for j in range(len(sorted_clusters)):
                if i == j :
                    continue

                column_clt = sorted_clusters[j]
                r2c = 0
                for time_row in row_clt.time_slides:
                    for time_col in column_clt.time_slides:
                        if time_row.overlaps(time_col):
                            r2c += 1

   #split no url clusters
    data_list_no_url = []
    for clt in wo_urls_list_poor:
        data_list_no_url.extend(clt.sessions)

    #one session in each cluster
    new_clusters_no_url = []
    for data in data_list_no_url:
        tmp = cert_cluster()
        tmp.sessions.append(data)
        tmp.update()
        new_clusters_no_url.append(tmp)


    #Similarity compution
    corr = 0


    for clt in new_clusters_no_url:
        sim_list = [time_sim(clt,item) for item in with_urls_list_poor]
#         sim_list = [cosine_similarity(np.array(clt.images).mean(axis=0).reshape(1,-1),np.array(item.images).mean(axis=0).reshape(1,-1))[0][0]
#                    for item in with_urls_list_poor]
        matched_idx = np.argmax(np.array(sim_list))
        if clt.label == with_urls_list_poor[matched_idx].label:
            corr += 1
    
        #gather to the roor clusters
        with_urls_list_poor[matched_idx].sessions.extend(clt.sessions)
    return with_urls_list_poor

    

In [25]:
from sklearn.metrics import confusion_matrix as conm
from sklearn.metrics import accuracy_score,precision_score,recall_score

def indicator_cal(y_true,y_pred):
    
#     y_pred = pred.flatten().tolist()
#     y_true = label.flatten().tolist()
    acc = accuracy_score(y_true,y_pred)
    pre = precision_score(y_true,y_pred,average='weighted')
    rec = recall_score(y_true,y_pred,average='weighted')
    f1 = (2.0*pre*rec/(pre+rec))
    
    return acc,pre,rec,f1


In [26]:
def acc_cal(with_urls_list_normal,with_urls_list_poor,matched_list):
    ground_truth = []
    predict = []

    for i in range(len(matched_list)): 
        tru_label = [item['label'] for item in with_urls_list_poor[matched_list[i]].sessions]
        prd_label = [with_urls_list_normal[i].label]*len(tru_label)
        predict.extend(prd_label)
        ground_truth.extend(tru_label)

    ground_truth  = np.array(ground_truth)
    predict = np.array(predict)

    return indicator_cal(ground_truth,predict)

#### Graph matching in same network

In [27]:
print("Cluster Processing...")
with_urls_list_normal, wo_urls_list_normal, urls_dict = clustering("data/InitTraffic_train.pkl")
with_urls_list_poor, wo_urls_list_poor, urls_dict = clustering("data/InitTraffic_test.pkl")
with_urls_list_normal, with_urls_list_poor = prepro(with_urls_list_normal,with_urls_list_poor)
print("Done!")
print("Matching...")
matching_matrix_normal,matching_matrix_poor,match_list = match_trans(with_urls_list_normal,with_urls_list_poor,urls_dict)
print("Done!")
print("Clustering isolate sessions...")
with_urls_list_poor = clt_final(with_urls_list_poor,wo_urls_list_poor)
print("Done!")
print("Calculating results...")
acc_cal(with_urls_list_normal,with_urls_list_poor,match_list)

Cluster Processing...
Done!
Matching...
Done!
Clustering isolate sessions...
Done!
Calculating results...


(0.9973591549295775,
 0.9974311779769526,
 0.9973591549295775,
 0.9973951651530485)

#### Graph matching in different networks

In [28]:
print("Cluster Processing...")
with_urls_list_normal, wo_urls_list_normal, urls_dict = clustering("data/InitTraffic.pkl")
with_urls_list_poor, wo_urls_list_poor, urls_dict = clustering("data/TestTraffic.pkl")
with_urls_list_normal, with_urls_list_poor = prepro(with_urls_list_normal,with_urls_list_poor)
print("Done!")
print("Matching...")
matching_matrix_normal,matching_matrix_poor,match_list = match_trans(with_urls_list_normal,with_urls_list_poor,urls_dict)
print("Done!")
print("Clustering isolate sessions...")
with_urls_list_poor = clt_final(with_urls_list_poor,wo_urls_list_poor)
print("Done!")
print("Calculating results...")
acc_cal(with_urls_list_normal,with_urls_list_poor,match_list)

Cluster Processing...
Done!
Matching...
Done!
Clustering isolate sessions...
Done!
Calculating results...


(0.9587188612099644,
 0.9654633852939702,
 0.9587188612099644,
 0.9620793030085361)